1. **Imports**:
   - We import necessary libraries and modules:
     - `pandas`: For data manipulation and analysis.
     - `numpy`: For numerical computations.
     - `MinMaxScaler` from `sklearn.preprocessing`: For feature scaling.
     - `train_test_split` from `sklearn.model_selection`: For splitting the dataset into training and testing sets.
     - `tensorflow` and its submodules: For building and training neural networks.
     - `matplotlib.pyplot`: For data visualization.

2. **Load Data**:
   - We load the cleaned dataset from the CSV file using `pd.read_csv()`.

3. **Data Preprocessing**:
   - We scale the input features using `MinMaxScaler` to ensure they are on a similar scale.
   - The dataset is split into training, validation, and test sets using `train_test_split()`.

4. **Model Architecture - DNN**:
   - We define the architecture of the Deep Neural Network (DNN) model using `Sequential()` from TensorFlow.keras.
   - The model consists of multiple dense (fully connected) layers with ReLU activation functions.
   - BatchNormalization layers are added to normalize the activations of the previous layer.
   - Dropout layers are added to mitigate overfitting by randomly dropping a fraction of the connections during training.

5. **Compile the Model**:
   - We compile the model using the Adam optimizer and Mean Squared Error (MSE) loss function.

6. **Callbacks**:
   - EarlyStopping: Monitor the validation loss and stop training if there is no improvement for a certain number of epochs to prevent overfitting.
   - ReduceLROnPlateau: Reduce the learning rate if the validation loss plateaus to help the model converge better.

7. **Model Training**:
   - We train the DNN model using the training data and validate it using the validation data.
   - The model is trained for a maximum of 50 epochs with a batch size of 64 samples.

8. **Evaluation**:
   - We evaluate the trained model's performance on the test set using the test data.
   - The test loss (MSE) is computed and printed to assess the model's generalization performance.

9. **Visualization**:
   - We plot the training and validation loss curves to visualize the training progress and identify potential overfitting.

10. **Prediction**:
    - Finally, we make predictions on the test set using the trained DNN model.

This code implements a Deep Neural Network model for energy consumption prediction in smart home environments, incorporating advanced techniques such as batch normalization, dropout regularization, and early stopping. It aims to achieve accurate and robust predictions while preventing overfitting and ensuring efficient training convergence. Adjustments to hyperparameters and architecture can be made based on experimentation and evaluation results.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

2024-02-15 21:20:04.655637: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the cleaned data
cleaned_data_file = '../data_cleaning_module/cleaned_data/HomeA-meter4_2016_updated.csv'
df = pd.read_csv(cleaned_data_file)

# Data Preprocessing
scaler = MinMaxScaler()
X = scaler.fit_transform(df.drop(columns=['Date & Time']))
y = df['use [kW]']

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Model Architecture - DNN
model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1)
])

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Define callbacks for early stopping and learning rate reduction
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), callbacks=callbacks)

# Evaluation
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)

# Plot training and validation loss curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Prediction
y_pred = model.predict(X_test)


Epoch 1/50
6285/6285 [==============================] - 17s 2ms/step - loss: 0.0392 - val_loss: 8.7401e-05 - lr: 0.0010
Epoch 2/50
6285/6285 [==============================] - 15s 2ms/step - loss: 4.9497e-04 - val_loss: 1.2830e-04 - lr: 0.0010
Epoch 3/50
6285/6285 [==============================] - 16s 3ms/step - loss: 3.0375e-04 - val_loss: 7.3324e-05 - lr: 0.0010
Epoch 4/50
6285/6285 [==============================] - 15s 2ms/step - loss: 2.1936e-04 - val_loss: 8.0542e-05 - lr: 0.0010
Epoch 5/50
6285/6285 [==============================] - 14s 2ms/step - loss: 1.7262e-04 - val_loss: 7.8706e-05 - lr: 2.0000e-04
Epoch 6/50
6285/6285 [==============================] - 15s 2ms/step - loss: 1.6960e-04 - val_loss: 1.0968e-04 - lr: 2.0000e-04
Epoch 7/50
6285/6285 [==============================] - 16s 3ms/step - loss: 1.6324e-04 - val_loss: 1.1667e-04 - lr: 2.0000e-04
Epoch 8/50
 570/1572 [=========>....................] - ETA: 1s - loss: 7.2847e-05